In [1]:
import pandas as pd
import numpy as np

In [2]:
net = pd.read_csv('netlist1.txt',sep = ' ',names =['component','from node','to node','value'] )
    

In [3]:
print(net)

  component  from node  to node  value
0        V1          1        0      4
1        V2          3        2      6
2        R1          1        2      1
3        R2          2        0      4
4        R3          3        0      2
5        I1          1        2      1


In [4]:
component_map = {'V':23,'R':24,'I':25}
net['component'] = net['component'].map(lambda x:component_map[x[0]])


In [5]:
print(net)

   component  from node  to node  value
0         23          1        0      4
1         23          3        2      6
2         24          1        2      1
3         24          2        0      4
4         24          3        0      2
5         25          1        2      1


In [6]:
net_arr = np.array(net)

In [7]:
print(net_arr)

[[23  1  0  4]
 [23  3  2  6]
 [24  1  2  1]
 [24  2  0  4]
 [24  3  0  2]
 [25  1  2  1]]


In [8]:
#Admitance matrix
no_node = net_arr[:,[1,2]].max()
no_branches = net_arr.shape[0]
def admitance_matrix(net_arr):
    
    Yn = np.zeros((no_node,no_node))
    res_branch = net_arr[net_arr[:,0] == 24,:]
    for i in range(res_branch.shape[0]):
        g = 1/res_branch[i,3]
        from_node = res_branch[i,1]
        to_node = res_branch[i,2]
        if from_node != 0:
            if to_node != 0:
                Yn[from_node - 1,to_node - 1] += -g
                Yn[to_node - 1,from_node - 1] += -g
                Yn[from_node - 1,from_node - 1] += g
                Yn[to_node - 1,to_node - 1] += g
            else:
                Yn[from_node - 1,from_node - 1] += g
                
        else:
            Yn[to_node - 1,to_node - 1] += g
            
    return Yn
            

In [9]:
Yn = admitance_matrix(net_arr)
print(Yn)


[[ 1.   -1.    0.  ]
 [-1.    1.25  0.  ]
 [ 0.    0.    0.5 ]]


In [10]:
vs_branch = net_arr[net_arr[:,0] == 23,:]
is_branch = net_arr[net_arr[:,0] == 25,:]
no_vs = vs_branch.shape[0]

In [11]:
print(vs_branch)

[[23  1  0  4]
 [23  3  2  6]]


In [12]:
print(net_arr[:,1])

[1 3 1 2 3 1]


In [13]:
#Vadj

no_vs  = vs_branch.shape[0]
def vadjjj(net_arr):
    vs_branch = net_arr[net_arr[:,0] == 23,:]
    #no_node = net_arr[:,[1,2]].max()
    vadjj = np.zeros((no_node,no_vs))
    for k in range(no_vs):
        for j in range(no_node):
            if vs_branch[k,1] == j+1:
                vadjj[j,k] = 1
            elif vs_branch[k,2] == j+1:
                vadjj[j,k] = -1
            else:
                vadjj[j,k] = 0
    return vadjj


In [14]:
zero_mat = np.zeros((no_vs,no_vs))

In [15]:
print(zero_mat)

[[0. 0.]
 [0. 0.]]


In [16]:
vadj = vadjjj(net_arr)
vadjT = vadj.T
print(vadjT)

[[ 1.  0.  0.]
 [ 0. -1.  1.]]


In [17]:
a = np.concatenate((Yn,vadj),axis = 1)
b = np.concatenate((vadjT,zero_mat),axis = 1)
Mna_mat = np.concatenate((a,b))

In [18]:
print(Mna_mat)

[[ 1.   -1.    0.    1.    0.  ]
 [-1.    1.25  0.    0.   -1.  ]
 [ 0.    0.    0.5   0.    1.  ]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.   -1.    1.    0.    0.  ]]


In [19]:
#injection vector
def inj_vv(net_arr):
    inj_v = np.zeros((no_node))
    for i in range(is_branch.shape[0]):
        node_from = is_branch[i,1]
        
        node_to = is_branch[i,2]
        
        if node_from != 0:
            inj_v[node_from - 1] += -is_branch[i,3]
        if node_to != 0:
            inj_v[node_to - 1] += is_branch[i,3]
    return inj_v
            
     

In [20]:
print(inj_vv(net_arr))

[-1.  1.  0.]


In [21]:
#incidence matrix

def cal_inc_mat(net_arr):
    inc_mat = np.zeros((no_node,no_branches))
    for i in range(0,no_branches):
        node_from = net_arr[i,1]
        node_to = net_arr[i,2]
        if node_from != 0:
            inc_mat[node_from - 1,i] = 1
        if node_to != 0:
            inc_mat[node_to - 1,i] = -1

    return inc_mat


In [22]:
#v rhs

vrhs = vs_branch[:,3]
print(vrhs)

[4 6]


In [23]:
#branch admitance matrix

def cal_bam(net_arr):
    yyn = np.zeros((no_branches,no_branches))
    for i in range(no_branches):
        if net_arr[i,0] == 24:
            yyn[i,i] = 1/net_arr[i,3]
    return yyn

In [24]:
rhs = np.concatenate((inj_vv(net_arr),vrhs))

In [25]:
#mna_computation

mna = np.linalg.solve(Mna_mat,rhs)
vn = mna[:no_node]
i_vs = mna[no_node:]
print(i_vs)

[-3.85714286 -3.57142857]


In [26]:
#branch voltage calculation
yb = cal_bam(net_arr)
vb = cal_inc_mat(net_arr).T@vn


In [27]:
#branch current calculation

ib = np.zeros(no_branches)
cs_mask = net_arr[:,0] == 25
vs_mask = net_arr[:,0] == 23
ib[cs_mask] = net_arr[cs_mask][:,3]
ib[vs_mask] = i_vs
ib += yb @ vb

In [28]:
print(ib)

[-3.85714286 -3.57142857  2.85714286  0.28571429  3.57142857  1.        ]


In [29]:
print(vb)

[4.         6.         2.85714286 1.14285714 7.14285714 2.85714286]
